## 10-fold Cross Validation on the Stanik dataset

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.metrics import matthews_corrcoef

# Check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 26.3 MB/s 
     |████████████████████████████████| 5.5 MB 88.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 7.6 MB 70.7 MB/s 
     |████████████████████████████████| 10.3 MB 77.5 MB/s 
     |████████████████████████████████| 1.9 MB 78.1 MB/s 
     |████████████████████████████████| 1.3 MB 75.1 MB/s 
     |████████████████████████████████| 451 kB 87.0 MB/s 
     |████████████████████████████████| 182 kB 86.5 MB/s 
     |████████████████████████████████| 182 kB 87.5 MB/s 
     |████████████████████████████████| 168 kB 84.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 168 kB 84.1 MB/s 
     |████████████████████████████████| 166 kB 90.0 MB/s 
     |████████████████████████████████| 166 kB 88.3 MB/s 
     |████████████████████████████

In [ ]:
# Import downsampled dataset
df = pd.read_excel('DATASET_downsampled_test2.xlsx')

df = df.drop(columns = ['Unnamed: 0'])

df.info()

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       2592 non-null   object
 1   kano_labels  2592 non-null   object
 2   labels       2592 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 60.9+ KB


In [ ]:
# Show the distribution of the labels
print(df['labels'].value_counts())

1    648
3    648
2    648
0    648
Name: labels, dtype: int64


In [ ]:
# Rename the columns so that simple transformers can handle them
train_data = df[['review','labels']]
train_data.columns = ['text', 'labels']
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2592 non-null   object
 1   labels  2592 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 40.6+ KB


In [ ]:
# Model implementation and k-fold cross validation with Simpletransformers

# Wrapper function to calculate f1-score with additional parameters
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average = None)

# Wrapper function to calculate recall with additional parameters
def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average = None)

# Wrapper function to calculate precision with additional parameters
def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average = None)

# K-fold cross validation with 10 folds
n = 10
kf = KFold(n_splits=n, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc = []
resultsf1 = []
resultsPrecision = []
resultsRecall = []

model_args = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create the ClassificationModel
model = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf.split(train_data):
  train_df = train_data.iloc[train_index]
  val_df = train_data.iloc[val_index]

  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)
  print(result)
  
  # Add scores to the respective lists
  resultsAcc.append(result['acc'])
  resultsf1.append(result['f1'])
  resultsPrecision.append(result['precision'])
  resultsRecall.append(result['recall'])

  print("results accuracy",resultsAcc)
  print(f"Mean-Precision: {sum(resultsAcc) / len(resultsAcc)}")
  print('results f1: ', resultsf1)
  print('results precision: ', resultsPrecision)
  print('results recall: ', resultsRecall)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

{'mcc': 0.9540591709439535, 'f1': array([0.97841727, 0.96721311, 0.97435897, 0.94285714]), 'acc': 0.9652509652509652, 'recall': array([1.        , 1.        , 0.95      , 0.91666667]), 'precision': array([0.95774648, 0.93650794, 1.        , 0.97058824]), 'eval_loss': 0.153783226555044}
results accuracy [0.7730769230769231, 0.8423076923076923, 0.8648648648648649, 0.888030888030888, 0.9420849420849421, 0.9884169884169884, 0.9575289575289575, 0.9806949806949807, 0.9652509652509652]
Mean-Precision: 0.9113619113619114
results f1:  [array([0.86524823, 0.73043478, 0.82706767, 0.65648855]), array([0.91935484, 0.84210526, 0.88333333, 0.72580645]), array([0.94890511, 0.84057971, 0.88372093, 0.77192982]), array([0.95798319, 0.83211679, 0.95726496, 0.82758621]), array([0.99224806, 0.91666667, 0.97333333, 0.86315789]), array([1.        , 0.97841727, 1.        , 0.97478992]), array([0.96969697, 0.94814815, 0.96078431, 0.95302013]), array([0.99280576, 0.96825397, 0.99259259, 0.96610169]), array([0.97

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/292 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/33 [00:00<?, ?it/s]

{'mcc': 0.9640881848168114, 'f1': array([0.98360656, 0.97014925, 0.98550725, 0.9516129 ]), 'acc': 0.972972972972973, 'recall': array([1.        , 0.98484848, 0.97142857, 0.93650794]), 'precision': array([0.96774194, 0.95588235, 1.        , 0.96721311]), 'eval_loss': 0.11885367529502025}
results accuracy [0.7730769230769231, 0.8423076923076923, 0.8648648648648649, 0.888030888030888, 0.9420849420849421, 0.9884169884169884, 0.9575289575289575, 0.9806949806949807, 0.9652509652509652, 0.972972972972973]
Mean-Precision: 0.9175230175230176
results f1:  [array([0.86524823, 0.73043478, 0.82706767, 0.65648855]), array([0.91935484, 0.84210526, 0.88333333, 0.72580645]), array([0.94890511, 0.84057971, 0.88372093, 0.77192982]), array([0.95798319, 0.83211679, 0.95726496, 0.82758621]), array([0.99224806, 0.91666667, 0.97333333, 0.86315789]), array([1.        , 0.97841727, 1.        , 0.97478992]), array([0.96969697, 0.94814815, 0.96078431, 0.95302013]), array([0.99280576, 0.96825397, 0.99259259, 0.966

In [ ]:
print("results accuracy",resultsAcc)
print(f"Mean-accuracy: {sum(resultsAcc) / len(resultsAcc)}")
print('results f1: ', resultsf1)
print('results precision: ', resultsPrecision)
print('results recall: ', resultsRecall)

results accuracy [0.7730769230769231, 0.8423076923076923, 0.8648648648648649, 0.888030888030888, 0.9420849420849421, 0.9884169884169884, 0.9575289575289575, 0.9806949806949807, 0.9652509652509652, 0.972972972972973]
Mean-accuracy: 0.9175230175230176
results f1:  [array([0.86524823, 0.73043478, 0.82706767, 0.65648855]), array([0.91935484, 0.84210526, 0.88333333, 0.72580645]), array([0.94890511, 0.84057971, 0.88372093, 0.77192982]), array([0.95798319, 0.83211679, 0.95726496, 0.82758621]), array([0.99224806, 0.91666667, 0.97333333, 0.86315789]), array([1.        , 0.97841727, 1.        , 0.97478992]), array([0.96969697, 0.94814815, 0.96078431, 0.95302013]), array([0.99280576, 0.96825397, 0.99259259, 0.96610169]), array([0.97841727, 0.96721311, 0.97435897, 0.94285714]), array([0.98360656, 0.97014925, 0.98550725, 0.9516129 ])]
results precision:  [array([0.84722222, 0.67741935, 0.79710145, 0.75438596]), array([0.91935484, 0.77108434, 0.98148148, 0.73770492]), array([0.92857143, 0.80555556, 

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

def PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1):

  np_resultsPrecision = np.array(resultsPrecision)
  np_resultsRecall = np.array(resultsRecall)
  np_resultsf1  = np.array(resultsf1)

  # Calculate the mean precision for each label
  df_resultsPrecision = pd.DataFrame(np_resultsPrecision, columns=list('BDIP'))
  precision_b_mean = df_resultsPrecision['B'].mean()
  precision_d_mean = df_resultsPrecision['D'].mean()
  precision_i_mean = df_resultsPrecision['I'].mean()
  precision_p_mean = df_resultsPrecision['P'].mean()

  # Calculate the mean recall for each label
  df_resultsRecall = pd.DataFrame(np_resultsRecall, columns=list('BDIP'))
  recall_b_mean = df_resultsRecall['B'].mean()
  recall_d_mean = df_resultsRecall['D'].mean()
  recall_i_mean = df_resultsRecall['I'].mean()
  recall_p_mean = df_resultsRecall['P'].mean()

  # Calculate the mean f1-score for each label
  df_resultsf1 = pd.DataFrame(np_resultsf1, columns=list('BDIP'))
  f1_b_mean = df_resultsf1['B'].mean()
  f1_d_mean = df_resultsf1['D'].mean()
  f1_i_mean = df_resultsf1['I'].mean()
  f1_p_mean = df_resultsf1['P'].mean()

  # Create dataframe with summarized results
  metrics = [{'B': precision_b_mean, 'D': precision_d_mean, 'I': precision_i_mean, 'P':precision_p_mean},
          {'B': recall_b_mean, 'D': recall_d_mean, 'I': recall_i_mean, 'P': recall_p_mean},
          {'B': f1_b_mean, 'D': f1_d_mean, 'I': f1_i_mean, 'P': f1_p_mean} ]
  train_df_metrics = pd.DataFrame(metrics, index=['Average precision',
                                'Average recall',
                                'Average F1-Score'])
  return train_df_metrics

PrecisionRecallF1Table(resultsPrecision, resultsRecall, resultsf1)

,B,D,I,P
Average precision,0.950938,0.872524,0.963913,0.886273
Average recall,0.971173,0.930030,0.926396,0.844053
Average F1-Score,0.960827,0.899408,0.943796,0.863335


## Testing on the Brunotte dataset

In [ ]:
# Import of 'Trainingskorpus_Final', which gives the final labeling
testset1 = pd.read_excel('Trainingskorpus_Final.xlsx')

testset1 = testset1.drop(columns = ['Unnamed: 0'])
testset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
dtypes: object(2)
memory usage: 25.5+ KB


In [ ]:
# Show the distribution of the labels
print(testset1['kano_labels'].value_counts())

B    1102
P     395
D      95
I      30
Name: kano_labels, dtype: int64


In [ ]:
# Import of 'Trainingskorpus_InitialLabels', which gives the initial labeling of two sometimes differing labels
testset2 = pd.read_excel('Trainingskorpus_InitialLabels.xlsx')

testset2 = testset2.drop(columns = ['Unnamed: 0'])
testset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1622 entries, 0 to 1621
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   review   1622 non-null   object
 1   label 1  1622 non-null   object
 2   label 2  1622 non-null   object
dtypes: object(3)
memory usage: 38.1+ KB


In [ ]:
# Merge the two sets so we have the final labels as well as the initial labels in the testset
testset = testset1.merge(testset2, left_on='review', right_on='review')
testset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1622 entries, 0 to 1621
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       1622 non-null   object
 1   kano_labels  1622 non-null   object
 2   label 1      1622 non-null   object
 3   label 2      1622 non-null   object
dtypes: object(4)
memory usage: 63.4+ KB


In [ ]:
# Add a column 'equal' which is True if the two initial labels are equal and false otherwise
equal = [None]*np.size(testset['review'])

# Add a column 'equal01' which stores binary values (1 if the two initial labels are equal, 0 otherwise) to calculate the phi-coefficient
equal01 = [None]*np.size(testset['review'])

# Add a column 'labeled_correctly01' which stores binary values (1 if the model correctly labeled the text, 0 otherwise) to calculate the phi-coefficient
labeled_correctly01 = [None]*np.size(testset['review'])

# Add a column 'prediction' which contains the model's predictions
prediction = [None]*np.size(testset['review'])

i = 0
for label1, label2 in zip(testset['label 1'], testset['label 2']):
  if(label1 == label2):
    equal[i] = True
    equal01[i] = 1
  else:
    equal[i] = False
    equal01[i] = 0
  i += 1
  
testset['equal'] = equal

In [ ]:
# Calculate the accuracy of the whole testset as well as 
# only the parts where the inital two labels are equal (accuracy_clear)
# or differ (accuracy_diff)

accuracy_clear = 0.
accuracy_diff = 0.
n_clear = 0
n_diff = 0
all_predicted_labels = []
predicted_labels_equal_initial_label = []
predicted_labels_diff_initial_label = []
true_label_equal = []
true_label_diff = []

# counter to go through all the reviews and fill the column 'labeled_correctly' with true if they have been labeled correctly, and false if not
i = 0

for label, review, equal in zip(testset['kano_labels'],  testset['review'], testset['equal']):
  
  test_prediction, raw_outputs = model.predict([review])
  
  predicted_label = ''

  if test_prediction[0] == 0:
    predicted_label = 'B'
    prediction[i] = 'B'
  if test_prediction[0] == 1:
    predicted_label = 'D'
    prediction[i] = 'D'
  if test_prediction[0] == 2:
    predicted_label = 'I'
    prediction[i] = 'I'
  if test_prediction[0] == 3:
    predicted_label = 'P'
    prediction[i] = 'P'

  all_predicted_labels.append(predicted_label)
  
  if equal:
    n_clear += 1
    predicted_labels_equal_initial_label.append(predicted_label)
    true_label_equal.append(label)
    if label == predicted_label:
      accuracy_clear += 1.

  if not equal:
    n_diff += 1
    predicted_labels_diff_initial_label.append(predicted_label)
    true_label_diff.append(label)
    if label == predicted_label:
      accuracy_diff += 1.

  if label == predicted_label:
    labeled_correctly01[i] = 1
  else:
    labeled_correctly01[i] = 0
  i+=1

testset['labeled_correctly'] = labeled_correctly01
testset['prediction'] = prediction

accuracy = (accuracy_clear + accuracy_diff) / (n_clear + n_diff)

accuracy_clear /= n_clear
accuracy_diff /= n_diff

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
testset.to_excel("Brunotte_with_predictions_test2_roberta.xlsx")  

In [ ]:
print('Accuracy Trainingskorpus:                                                  ', round(accuracy*100, 2), '%,  n =', (n_clear + n_diff))
print('Accuracy Trainingskorpus only on the data that coincides on initial labels:', round(accuracy_clear*100, 2), '%,  n =', n_clear)
print('Accuracy Trainingskorpus only on the data that differs on initial labels:  ', round(accuracy_diff*100, 2), '%,  n = ', n_diff)

Accuracy Trainingskorpus:                                                   70.22 %,  n = 1622
Accuracy Trainingskorpus only on the data that coincides on initial labels: 75.05 %,  n = 1383
Accuracy Trainingskorpus only on the data that differs on initial labels:   42.26 %,  n =  239


In [ ]:
# Calculate precision, recall and f1-score for each label

test_df_metrics = precision_recall_fscore_support(testset['kano_labels'], all_predicted_labels, average=None,labels=['B', 'D', 'I', 'P'])

print('Precision B: ', test_df_metrics[0][0])
print('Precision D: ', test_df_metrics[0][1])
print('Precision I: ', test_df_metrics[0][2])
print('Precision P: ', test_df_metrics[0][3])

print('Recall B: ', test_df_metrics[1][0])
print('Recall D: ', test_df_metrics[1][1])
print('Recall I: ', test_df_metrics[1][2])
print('Recall P: ', test_df_metrics[1][3])

print('F1-score B: ', test_df_metrics[2][0])
print('F1-score D: ', test_df_metrics[2][1])
print('F1-score I: ', test_df_metrics[2][2])
print('F1-score P: ', test_df_metrics[2][3])

Precision B:  0.8934337997847147
Precision D:  0.39106145251396646
Precision I:  0.0
Precision P:  0.4649805447470817
Recall B:  0.7531760435571688
Recall D:  0.7368421052631579
Recall I:  0.0
Recall P:  0.6050632911392405
F1-score B:  0.8173313638601674
F1-score D:  0.510948905109489
F1-score I:  0.0
F1-score P:  0.5258525852585259


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Calculates the phi-coefficient of the labels being predicted correctly and the researchers being 

print('MCC for initial agreement of raters and correct or false labeling',matthews_corrcoef(labeled_correctly01, equal01))

MCC for initial agreement of raters and correct or false labeling 0.2542024944086244


## 10-fold Cross Validation on a combination of the Stanik dataset and the Brunotte dataset

In [ ]:
# Import combined downsampled dataset 
df_combined = pd.read_excel('DATASET_Trainingskorpus_combined_test2.xlsx')

df_combined = df_combined.drop(columns = ['Unnamed: 0'])

# Rename column so that simple transformers can handle it
df_combined = df_combined.rename(columns = {'review': 'text'})
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2972 entries, 0 to 2971
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         2972 non-null   object
 1   kano_labels  2972 non-null   object
 2   labels       2972 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 69.8+ KB


In [ ]:
# Show the distribution of the labels
print(df_combined['labels'].value_counts())

1    743
0    743
2    743
3    743
Name: labels, dtype: int64


In [ ]:
# 10-fold cross validation on the combined dataset

n2 = 10
kf2 = KFold(n_splits=n2, random_state = 42, shuffle = True)

# Lists to store the values of accuracy, f1-score, precision and recall for all 10 folds
resultsAcc2 = []
resultsf12 = []
resultsPrecision2 = []
resultsRecall2 = []

model_args2 = ClassificationArgs(
    num_train_epochs=1, 
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    overwrite_output_dir= True
)

# Create a ClassificationModel
model2 = ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=4,
    args=model_args,
    #use_cuda=False
) 

for train_index, val_index in kf2.split(df_combined):
  train_df2 = df_combined.iloc[train_index]
  val_df2 = df_combined.iloc[val_index]

  model2.train_model(df_combined)
  result2, model_outputs2, wrong_predictions2 = model2.eval_model(val_df2, f1=f1_multiclass, acc=accuracy_score, recall=recall_multiclass, precision=precision_multiclass)

  print(result2)
  # Add scores to the respective lists
  resultsAcc2.append(result2['acc'])
  resultsf12.append(result2['f1'])
  resultsPrecision2.append(result2['precision'])
  resultsRecall2.append(result2['recall'])

  print("results accuracy",resultsAcc2)
  print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
  print('results f1: ', resultsf12)
  print('results precision: ', resultsPrecision2)
  print('results recall: ', resultsRecall2)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 0.7539861060239565, 'f1': array([0.85714286, 0.79104478, 0.85714286, 0.77248677]), 'acc': 0.8154362416107382, 'recall': array([0.89552239, 0.85483871, 0.78082192, 0.76041667]), 'precision': array([0.82191781, 0.73611111, 0.95      , 0.78494624]), 'eval_loss': 0.5139611645748741}
results accuracy [0.8154362416107382]
Mean-accuracy: 0.8154362416107382
results f1:  [array([0.85714286, 0.79104478, 0.85714286, 0.77248677])]
results precision:  [array([0.82191781, 0.73611111, 0.95      , 0.78494624])]
results recall:  [array([0.89552239, 0.85483871, 0.78082192, 0.76041667])]


...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/372 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Evaluation:   0%|          | 0/38 [00:00<?, ?it/s]

{'mcc': 1.0, 'f1': array([1., 1., 1., 1.]), 'acc': 1.0, 'recall': array([1., 1., 1., 1.]), 'precision': array([1., 1., 1., 1.]), 'eval_loss': 0.0017153981484864887}
results accuracy [0.8154362416107382, 0.889261744966443, 0.936026936026936, 0.9528619528619529, 0.9696969696969697, 0.9764309764309764, 0.9797979797979798, 0.9797979797979798, 0.9865319865319865, 1.0]
Mean-accuracy: 0.9485842767721964
results f1:  [array([0.85714286, 0.79104478, 0.85714286, 0.77248677]), array([0.93975904, 0.8358209 , 0.96296296, 0.79104478]), array([0.95588235, 0.9375    , 0.97402597, 0.875     ]), array([0.95172414, 0.95424837, 0.99310345, 0.91390728]), array([0.98717949, 0.96774194, 0.98571429, 0.93706294]), array([0.9704142 , 0.98113208, 1.        , 0.95890411]), array([1.        , 0.96296296, 0.99300699, 0.95238095]), array([0.98360656, 0.96815287, 0.98630137, 0.98224852]), array([0.98630137, 0.98666667, 0.99270073, 0.98136646]), array([1., 1., 1., 1.])]
results precision:  [array([0.82191781, 0.736111

In [ ]:
print("results accuracy",resultsAcc2)
print(f"Mean-accuracy: {sum(resultsAcc2) / len(resultsAcc2)}")
print('results f1: ', resultsf12)
print('results precision: ', resultsPrecision2)
print('results recall: ', resultsRecall2)

results accuracy [0.8154362416107382, 0.889261744966443, 0.936026936026936, 0.9528619528619529, 0.9696969696969697, 0.9764309764309764, 0.9797979797979798, 0.9797979797979798, 0.9865319865319865, 1.0]
Mean-accuracy: 0.9175230175230176
results f1:  [array([0.85714286, 0.79104478, 0.85714286, 0.77248677]), array([0.93975904, 0.8358209 , 0.96296296, 0.79104478]), array([0.95588235, 0.9375    , 0.97402597, 0.875     ]), array([0.95172414, 0.95424837, 0.99310345, 0.91390728]), array([0.98717949, 0.96774194, 0.98571429, 0.93706294]), array([0.9704142 , 0.98113208, 1.        , 0.95890411]), array([1.        , 0.96296296, 0.99300699, 0.95238095]), array([0.98360656, 0.96815287, 0.98630137, 0.98224852]), array([0.98630137, 0.98666667, 0.99270073, 0.98136646]), array([1., 1., 1., 1.])]
results precision:  [array([0.82191781, 0.73611111, 0.95      , 0.78494624]), array([0.91764706, 0.8358209 , 0.975     , 0.8030303 ]), array([0.95588235, 0.9375    , 0.97402597, 0.875     ]), array([0.90789474, 0.

In [ ]:
# Calculate the average precision, recall and f1-scores of the 10 folds for each label

PrecisionRecallF1Table(resultsPrecision2, resultsRecall2, resultsf12)

,B,D,I,P
Average precision,0.955523,0.930252,0.987025,0.922565
Average recall,0.971845,0.948212,0.963717,0.910613
Average F1-Score,0.963201,0.938527,0.974496,0.916440
